In [1]:
import numpy as np
import pandas as pd
import stateEngine
from sklearn.linear_model import ElasticNetCV
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
corpus = pd.read_csv("content/output.csv")
corpus.head()

,turn,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,property_9,...,jail_card_1,jail_card_2,agent_position_1,agent_position_2,agent_cash_1,agent_cash_2,agent_1_owed,agent_1_debt,agent_2_owed,agent_2_debt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,1300.0,1500.0,0,0,0,0
1,1,0,0,0,-1,0,0,0,0,0,...,0,0,4,3,1300.0,1440.0,0,0,0,0
2,2,0,0,0,-1,0,0,0,0,0,...,0,0,9,3,1180.0,1440.0,0,0,0,0
3,3,0,0,0,-1,0,0,0,0,0,...,0,0,9,11,1180.0,1300.0,0,0,0,0
4,4,0,0,0,-1,0,0,0,0,0,...,0,0,12,11,1030.0,1300.0,0,0,0,0


In [3]:
features = []
game = 0
prev = None
count = 0
for r in corpus.values:
    x = r.tolist()
    v = [x[0], x[1:43], x[43:45], x[45:47], x[47:]]
    s = stateEngine.State(1, v)
    
    turnNo = s.currentTurnNumber()
    
    if(prev is None or turnNo < prev):
        game += 1
        
    features.append(np.append(np.array([game]), np.append(r, s.extract_features())))
    
    prev = turnNo
    
output = np.array(features)
headers = ['game'] + corpus.columns.values.tolist() + stateEngine.State(1, [[]]).extract_headers()
corpus = pd.DataFrame(output, columns=headers)
corpus = corpus[corpus.columns.drop(list(corpus.filter(regex='property')))]
corpus.shape

(113554, 26)

In [4]:
pd.set_option('display.max_columns', 1000)
corpus.head()

,game,turn,jail_card_1,jail_card_2,agent_position_1,agent_position_2,agent_cash_1,agent_cash_2,agent_1_owed,agent_1_debt,agent_2_owed,agent_2_debt,agent_netwealth_1,agent_netwealth_2,agent_liqassets_1,agent_liqassets_2,agent_propcount_1,agent_propcount_2,agent_monopoly_1,agent_monopoly_2,total_wealth,total_propoerty_count,agent_propratio_1,agent_propratio_2,agent_bldgratio_1,agent_bldgratio_2
0,1.0,0.0,0.0,0.0,4.0,0.0,1300.0,1500.0,0.0,0.0,0.0,0.0,0.464286,0.535714,0.464286,0.535714,0.0,0.0,0.0,0.0,2800.0,0.0,0.000000,0.000000,0.0,0.0
1,1.0,1.0,0.0,0.0,4.0,3.0,1300.0,1440.0,0.0,0.0,0.0,0.0,0.464286,0.535714,0.469314,0.530686,0.0,1.0,0.0,0.0,2800.0,1.0,0.000000,1.000000,0.0,0.0
2,1.0,2.0,0.0,0.0,9.0,3.0,1180.0,1440.0,0.0,0.0,0.0,0.0,0.464286,0.535714,0.457565,0.542435,1.0,1.0,0.0,0.0,2800.0,2.0,0.500000,0.500000,0.0,0.0
3,1.0,3.0,0.0,0.0,9.0,11.0,1180.0,1300.0,0.0,0.0,0.0,0.0,0.464286,0.535714,0.469697,0.530303,1.0,2.0,0.0,0.0,2800.0,3.0,0.333333,0.666667,0.0,0.0
4,1.0,4.0,0.0,0.0,12.0,11.0,1030.0,1300.0,0.0,0.0,0.0,0.0,0.464286,0.535714,0.454191,0.545809,2.0,2.0,0.0,0.0,2800.0,4.0,0.500000,0.500000,0.0,0.0


In [5]:
game_list = corpus['game'].unique()
print(game_list)

[    1.     2.     3.     4.     5.     6.     7.     8.     9.    10.
    11.    12.    13.    14.    15.    16.    17.    18.    19.    20.
    21.    22.    23.    24.    25.    26.    27.    28.    29.    30.
    31.    32.    33.    34.    35.    36.    37.    38.    39.    40.
    41.    42.    43.    44.    45.    46.    47.    48.    49.    50.
    51.    52.    53.    54.    55.    56.    57.    58.    59.    60.
    61.    62.    63.    64.    65.    66.    67.    68.    69.    70.
    71.    72.    73.    74.    75.    76.    77.    78.    79.    80.
    81.    82.    83.    84.    85.    86.    87.    88.    89.    90.
    91.    92.    93.    94.    95.    96.    97.    98.    99.   100.
   101.   102.   103.   104.   105.   106.   107.   108.   109.   110.
   111.   112.   113.   114.   115.   116.   117.   118.   119.   120.
   121.   122.   123.   124.   125.   126.   127.   128.   129.   130.
   131.   132.   133.   134.   135.   136.   137.   138.   139.   140.
   141

In [6]:
#Group by the game
game_list = corpus.groupby('game')
games = [group for _, group in game_list if group.values.shape[0] > 50]

In [7]:
regressor_vector_p1 = []
regressor_vector_p2 = []
test_set_x_1 = []
test_set_y_1 = []
test_set_x_2 = []
test_set_y_2 = []

In [ ]:
for game in games:
    t1 = game['agent_netwealth_1'].values
    t2 = game['agent_netwealth_2'].values
    
    X1_train, X1_test, y1_train, y1_test = train_test_split(game.drop('agent_netwealth_1', axis=1).values, t1, test_size=0.15)
    X2_train, X2_test, y2_train, y2_test = train_test_split(game.drop('agent_netwealth_2', axis=1).values, t2, test_size=0.15)
    
    e_net1 = ElasticNetCV(cv=10, random_state=0, max_iter=1e5)
    e_net1.fit(X1_train, y1_train)
    print(e_net1.score(X1_test, y1_test))
    
    
    e_net2 = ElasticNetCV(cv=10, random_state=0, max_iter=1e5)
    e_net2.fit(X2_train, y2_train)
    print(e_net2.score(X2_test, y2_test))
    
    test_set_x_2.append(X2_test)
    test_set_y_2.append(y2_test)
    test_set_x_1.append(X1_test)
    test_set_y_1.append(y1_test)
    
    regressor_vector_p1.append(e_net1.coef_)
    regressor_vector_p2.append(e_net2.coef_)
    
    del X1_train, X1_test, y1_train, y1_test, X2_train, X2_test, y2_train, y2_test

0.333163124142
0.707000449929
0.949720950632
0.924216964042
0.59082591985
0.690309679
0.714607554456
0.7241684777
0.446800618285
0.597734350057
0.68730854551
0.562460122072
0.73888626636
0.855917116936
0.75095741329
0.760571219156
0.89408469115
0.8924016354
0.931139399417
0.923100996005
0.439746589984
0.575955962557
0.725296322559
0.766209251138
0.804513549818
0.817194523844
0.515856375756
0.666562281432
0.721786981388
0.653497300281
0.590519313048
0.671121362753
0.667559412807
0.742592017344
0.85929447847
0.866618975795
0.72297357693
0.680356205336
0.762212710045
0.70497288784
0.516556827169
0.662528151615
0.949300687601
0.89855661007
0.817879600117
0.837603112218
0.70483140847
0.556034007343
0.973373123709
0.975265673566
0.830673426629
0.885162455404
0.590382047534
0.475470630881
0.75464005803
0.596779654116
0.957780750235
0.979603490383
0.848136643654
0.892418770246
0.929513697095
0.936505703319
0.934086147778
0.864290666349
0.89346413695
0.944170611276
0.939441588204
0.937985754796

0.934371853348
0.954066978886
0.890662291875
0.89838851972
0.911168376132
0.970679172029
0.983750882749
0.841378025213
0.768730578759
0.641589109343
0.894392282401
0.724922135161
0.847721374037
-1.02282407128
0.805281668296
0.932245899633
0.925250153511
0.867498634079
0.838334615971
0.840210802223
0.893953430326
0.785595558025
0.861377271591
0.913076480816
0.900918961667
0.982672393515
0.981896419778
0.831871550287
0.886997045184
0.941710042168
0.923840506676
0.936425943056
0.94870461731
0.977645666905
0.949727987264
0.650430620275
0.478912377196
0.827594297129
0.730691580377
0.96537594592
0.965485413024
0.864721861876
0.875027201282
0.758462298741
0.242674236834
0.480946132748
0.346151842957
0.878104349613
0.823783273312
0.857473833354
0.66042467141
0.765326930299
0.644617025403
0.838562765687
0.833024508822
0.910942779789
0.891109703672
-0.0471342677398
-0.47239612143
0.879451450564
0.882226883018
0.859680783935
0.977691339676
0.718302828824
0.426500505817
0.746542448095
0.7273871321

0.939138643894
0.923924064036
0.824617085737
0.96310490499
0.953817524528
0.888858425576
0.870257327792
0.849527453944
0.704245974637
0.828300014677
0.790395888461
0.952921048314
0.960912388335
0.942956559098
0.950280701081
0.98571487516
0.97686725184
0.83280712733
0.748540900533
0.94815980494
0.926128545715
-0.269985693883
0.0981631406715
0.967061233015
0.952178970175
0.92131018454
0.917084505739
0.875919215561
0.883772828981
0.569631205631
0.824494695767
0.905176424286
0.931571970264
0.715019299288
0.264713435014
0.866662042025
0.894992578727
0.860802450439
0.918826979497
0.674178752265
0.838150432463
0.548265535453
0.656873113659
-0.181212349019
0.897068467437
0.881758204148
0.810028881738
0.966135210778
0.875852180861
0.260809601817
0.523938445471
0.892331385955
0.889616605059
0.394270177057
0.608105095057
0.908152867634
0.93390456704
0.832446269321
0.783395562081
0.64333334944
0.670596552061
0.00786915882085
0.142279073176
0.808418202002
0.925473098065
0.891725875138
0.93319111949

In [ ]:
weight1 = np.mean(np.array(regressor_vector_p1), axis=0)
weight2 = np.mean(np.array(regressor_vector_p2), axis=0)

In [ ]:
# np.concatenate(np.array(test_set_y_1))
y_hat_1 = np.dot(np.concatenate(np.array(test_set_x_1)), weight1.reshape(-1, 1)).flatten()
mean_squared_error(np.concatenate(np.array(test_set_y_1)), y_hat_1)

In [ ]:
y_hat_2 = np.dot(np.concatenate(np.array(test_set_x_2)), weight1.reshape(-1, 1)).flatten()
mean_squared_error(np.concatenate(np.array(test_set_y_2)), y_hat_2)